In [2]:
import json
import torch
from PIL import Image
import torchvision.transforms as transforms
import numpy as np
import json
import requests
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f'Using {device} for inference')


Using cuda for inference


In [1]:
import argparse
import os
import torch 
import numpy as np
import torchvision
import torch.nn as nn
from torchvision.utils import save_image
import matplotlib.pyplot as plt
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data.dataloader import DataLoader
from torch.nn.modules.utils import _pair
import torch.nn.functional as F
import math
import copy
import time
import pdb
from statistics import mean
from torch.utils.tensorboard import SummaryWriter
from pathlib import Path
from PIL import Image
import gc
import h5py
import glob
from sklearn.model_selection import KFold
from sklearn.linear_model import Ridge
from scipy.stats.stats import pearsonr
from sklearn.decomposition import PCA

C:\Users\aswil\AppData\Local\Temp\ipykernel_39588\3881796793.py:27: DeprecationWarning: Please use `pearsonr` from the `scipy.stats` namespace, the `scipy.stats.stats` namespace is deprecated.
  from scipy.stats.stats import pearsonr


In [ ]:
resnet50 = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_resnet50', pretrained=True)
utils = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_convnets_processing_utils')

resnet50.eval().to(device)

In [44]:
for name, module in resnet50.named_modules():
    print(name)


conv1
bn1
relu
maxpool
layers
layers.0
layers.0.0
layers.0.0.conv1
layers.0.0.bn1
layers.0.0.conv2
layers.0.0.bn2
layers.0.0.conv3
layers.0.0.bn3
layers.0.0.relu
layers.0.0.downsample
layers.0.0.downsample.0
layers.0.0.downsample.1
layers.0.1
layers.0.1.conv1
layers.0.1.bn1
layers.0.1.conv2
layers.0.1.bn2
layers.0.1.conv3
layers.0.1.bn3
layers.0.1.relu
layers.0.2
layers.0.2.conv1
layers.0.2.bn1
layers.0.2.conv2
layers.0.2.bn2
layers.0.2.conv3
layers.0.2.bn3
layers.0.2.relu
layers.1
layers.1.0
layers.1.0.conv1
layers.1.0.bn1
layers.1.0.conv2
layers.1.0.bn2
layers.1.0.conv3
layers.1.0.bn3
layers.1.0.relu
layers.1.0.downsample
layers.1.0.downsample.0
layers.1.0.downsample.1
layers.1.1
layers.1.1.conv1
layers.1.1.bn1
layers.1.1.conv2
layers.1.1.bn2
layers.1.1.conv3
layers.1.1.bn3
layers.1.1.relu
layers.1.2
layers.1.2.conv1
layers.1.2.bn1
layers.1.2.conv2
layers.1.2.bn2
layers.1.2.conv3
layers.1.2.bn3
layers.1.2.relu
layers.1.3
layers.1.3.conv1
layers.1.3.bn1
layers.1.3.conv2
layers.1.3.bn

In [17]:
def open_images():
    ''' 
    (int, dict) -> NoneType
    
    Preprocess the data images and put them in a dictionary
    '''
    images = []
    for i in range (0, 5):
        image = fr"C:\Users\aswil\Desktop\College\Internships\Bashivan_Lab\Projects\glass_dots\glass_dots_code\hyperbolic_img\{i}.jpg"
        

        with open(image, 'rb') as file:
            print(image)

   
            img = Image.open(image).resize((224, 224))
    
            img_norm = np.array(img, dtype=np.float32) / 255.
        
            img_norm = (img_norm - [0.485, 0.456, 0.406]) / [0.229, 0.224, 0.225]
            
            images.append(np.transpose(img_norm, (2, 0, 1)))
        
             
    return images
    

In [18]:
images_np = np.array(open_images(), dtype=np.float32)

C:\Users\aswil\Desktop\College\Internships\Bashivan_Lab\Projects\glass_dots\glass_dots_code\hyperbolic_img\0.jpg
C:\Users\aswil\Desktop\College\Internships\Bashivan_Lab\Projects\glass_dots\glass_dots_code\hyperbolic_img\1.jpg
C:\Users\aswil\Desktop\College\Internships\Bashivan_Lab\Projects\glass_dots\glass_dots_code\hyperbolic_img\2.jpg
C:\Users\aswil\Desktop\College\Internships\Bashivan_Lab\Projects\glass_dots\glass_dots_code\hyperbolic_img\3.jpg
C:\Users\aswil\Desktop\College\Internships\Bashivan_Lab\Projects\glass_dots\glass_dots_code\hyperbolic_img\4.jpg


In [19]:
dataloader = torch.utils.data.DataLoader(torch.from_numpy(images_np), batch_size=20, shuffle=False)

In [20]:
def get_activation(name, activation):        
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook

In [21]:
def get_activation_list(model, data_loader):
    all_activation = {}
    
    for i, data in enumerate(data_loader):
        _ = model(data.to(device))
        if len(all_activation) == 0: 
            for k in activation.keys(): 
                all_activation[k] = activation[k].cpu().numpy()
        else: 
            for k in activation.keys():
                all_activation[k] = np.concatenate([all_activation[k], activation[k].cpu().numpy()])
    
    return all_activation

In [66]:
activation = {}
model_layer_names = ["layers.0.0", "layers.0.1", "layers.0.2", "layers.1.0", "layers.1.1", "layers.1.2", "layers.1.3", "layers.2.0", "layers.2.1", "layers.2.2", "layers.2.3", "layers.2.4", "layers.2.5", "layers.3.0", "layers.3.1", "layers.3.2" ]
x = 1

for name, module in resnet50.named_modules():
    if name in model_layer_names:
        #print(name)
        module.register_forward_hook(get_activation(name, activation))
all_activations = get_activation_list(resnet50, dataloader)
print(all_activations)
print(all_activations['layers.0.0'].shape)

{'layers.0.0': array([[[[0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.00841314],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         ...,
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.0861911 ]],

        [[0.86527056, 0.7250047 , 0.72604793, ..., 0.726071  ,
          0.7364809 , 0.60513496],
         [0.93735695, 0.5023328 , 0.54739904, ..., 0.54429543,
          0.51561415, 0.46737266],
         [0.98058033, 0.54033375, 0.5547888 , ..., 0.55441046,
          0.55024004, 0.4930737 ],
         ...,
         [0.99436957, 0.5493041 , 0.55922544, ..., 0.5612446 ,
          

In [67]:
seed=10
PCA_activations={}
for layer in ["layers.0.0", "layers.0.1", "layers.0.2", "layers.1.0", "layers.1.1", "layers.1.2", "layers.1.3", "layers.2.0", "layers.2.1", "layers.2.2", "layers.2.3", "layers.2.4", "layers.2.5", "layers.3.0", "layers.3.1", "layers.3.2" ]:
    act_now = all_activations[layer]
    pca=PCA(random_state=seed)
    pcaact_now = pca.fit_transform(act_now.reshape(185,-1))
    PCA_activations[layer] = pcaact_now

ValueError: cannot reshape array of size 4014080 into shape (185,newaxis)